In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import datetime


In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
year = 1995 # Year we're currently testing.
html = requests.get('https://en.wikipedia.org/wiki/List_of_Billboard_Hot_100_top-ten_singles_in_' + str(year),cookies={ 'sessionid': '123..'},headers=headers).text
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify()[0:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Billboard Hot 100 top-ten singles in 1995 - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XgWP5wpAME0AADcz6oMAAABH","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Billboard_Hot_100_top-ten_singles_in_1995","wgTitle":"List of Billboard Hot 100 top-ten singles in 1995","wgCurRevisionId":899235184,"wgRevisionId":899235184,"wgArticleId":40416612,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":

In [4]:
my_table = soup.find('table',{'class':'wikitable sortable'})
type(my_table)

bs4.element.Tag

In [5]:
Titles = []
tableheaderincluded = True

# The following code mines the first table found on the Wikipedia page and extracts only table elements that do not correspond to a month and date. We don't care about the exact month and day that the song achieved Top 10 glory or their peek date. Furthermore, the way Wikipedia's tables are formatted, there are occasionally multiple songs that were all entered on the same day, and that date is listed only once, which creates problems for our script later down the road.

try:
    tds = my_table.findAll('td')
except AttributeError:
    my_table = soup.find('table',{'class':'wikitable'})
    tableheaderincluded = False
    tds = my_table.findAll('td')

for td in tds:
    try:
        d = datetime.datetime.strptime(td.get_text()[:-1], "%B %d") # Check if the current string is a Month Day format.
    except ValueError:
        try:
            d = datetime.datetime.strptime(td.get_text()[:td.get_text().rfind('(')-1], "%B %d") # Catch a rare case where a song from the previous year charted again.
        except ValueError:
            try:
                d = datetime.datetime.strptime(td.get_text()[:td.get_text().rfind('[')-1], "%B %d") # Catch a rare case where a song's top ten entry date has an annotation.
            except ValueError:
                if td.get_text()[0] != '[' and td.get_text()[0] and td.get_text()[0].strip(): # Drop annotations column.
                    Titles.append(td.get_text()[:-1]) # Drop the newline character present at the end of every single entry.



In [6]:
print(Titles)

['"Creep"', 'TLC', '1', '20', '"Before I Let You Go"', 'Blackstreet', '7', '8', '"I\'m the Only One"', 'Melissa Etheridge', '8', '5', '"Tootsee Roll"', '69 Boyz', '8', '3', '"Take a Bow"', 'Madonna', '1', '15', '"Sukiyaki"', '4 P.M.', '8', '6', '"You Gotta Be"', "Des'ree", '5', '10', '"Baby"', 'Brandy', '4', '9', '"Candy Rain"', 'Soul for Real', '2', '12', '"Hold My Hand"', 'Hootie & The Blowfish', '10', '3', '"If You Love Me"', 'Brownstone', '8', '5', '"Strong Enough"', 'Sheryl Crow', '5', '11', '"Big Poppa"', 'The Notorious B.I.G', '6', '7', '"Red Light Special"', 'TLC', '2', '12', '"Run Away"', 'Real McCoy', '3', '8', '"This Is How We Do It"', 'Montell Jordan', '1', '14', '"I Know"', 'Dionne Farris', '4', '10', '"Freak like Me"', 'Adina Howard', '2', '12', '"Keep Their Heads Ringin\'"', 'Dr. Dre', '10', '1', '"Dear Mama"', '2Pac', '9', '4', '"I Believe"', 'Blessid Union of Souls', '8', '8', '"Water Runs Dry"', 'Boyz II Men', '2', '16', '"Have You Ever Really Loved a Woman?"', 'Bryan

In [7]:
nptitles = np.asarray(Titles).copy()

In [8]:
len(nptitles)

232

In [9]:
numsongs = int(np.prod(nptitles.shape)/4) # Find out how many songs are in this list.
reshaped = nptitles.reshape(numsongs,4) # Reshape the array based on the number of songs and the number of columns.

In [10]:
songdata = pd.DataFrame(reshaped)
songdata.columns = ['Title','Artist','Peak','Number of weeks in top ten']
songdata['Year'] = year * numsongs
songdata['Title'] = [i[1:i.rfind('\"')] for i in songdata['Title']] # Drops quotation marks and annotations in Title.
cleanweeks = []
for i in songdata['Number of weeks in top ten']: # This code searches the number of weeks in top ten column for asterisks and eliminates them. This is only a 
    try:                                         # problem in the current year.
        cleanweeks.append(int(i[:i.rindex('*')]))
    except ValueError:
        cleanweeks.append(int(i))

songdata['Number of weeks in top ten'] = cleanweeks
del songdata['Peak'] # This will not be relevant to our analysis.

In [11]:
songdata

,Title,Artist,Number of weeks in top ten,Year
0,Creep,TLC,20,2019
1,Before I Let You Go,Blackstreet,8,2019
2,I'm the Only One,Melissa Etheridge,5,2019
3,Tootsee Roll,69 Boyz,3,2019
4,Take a Bow,Madonna,15,2019
5,Sukiyaki,4 P.M.,6,2019
6,You Gotta Be,Des'ree,10,2019
7,Baby,Brandy,9,2019
8,Candy Rain,Soul for Real,12,2019
9,Hold My Hand,Hootie & The Blowfish,3,2019


In [13]:
songdata.to_csv(path_or_buf='data/' + year + 'top10songs.csv',index=False) # Export our sample data table.